In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import srgan
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
y = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 2048)
partitioned_y = malaya_speech.tf_featurization.pad_and_partition(y, 8192)
model = srgan.Model(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
model.logits, partitioned_y
model.logits.set_shape((None, 8192, 1))

In [6]:
hr_output = srgan.Discriminator(partitioned_y)
sr_output = srgan.Discriminator(model.logits)

Tensor("flatten/Reshape:0", shape=(?, 262144), dtype=float32)
Tensor("flatten_1/Reshape:0", shape=(?, 262144), dtype=float32)


In [14]:
hr_output.logits

<tf.Tensor 'dense_1/Sigmoid:0' shape=(?, 1) dtype=float32>

In [7]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(y)[0]]
logits

<tf.Tensor 'strided_slice_9:0' shape=(?, 1) dtype=float32>

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
y_lr, sr_lr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 11025)
y_hr, sr_hr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 44100)

In [10]:
%%time

y_ = sess.run(logits, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

CPU times: user 1min 12s, sys: 4.03 s, total: 1min 16s
Wall time: 14.1 s


(517734, 1)

In [11]:
%%time

y_ = sess.run(model.logits, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

CPU times: user 1min 14s, sys: 4 s, total: 1min 18s
Wall time: 12.5 s


(64, 8192, 1)

In [12]:
%%time

y_ = sess.run(partitioned_y, feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})
y_.shape

CPU times: user 50.6 ms, sys: 16.4 ms, total: 67.1 ms
Wall time: 56.4 ms


(64, 8192, 1)

In [14]:
y_[0].shape

(1,)

In [15]:
y_[1].shape

(1,)

In [16]:
y_.reshape((-1)).shape

(517734,)

In [17]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 44100)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, 'test/model.ckpt')

In [15]:
!ls -lh test

total 59248
-rw-r--r--  1 huseinzolkepli  staff    77B Jan 13 20:29 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    28M Jan 13 20:29 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   7.7K Jan 13 20:29 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   1.0M Jan 13 20:29 model.ckpt.meta


In [20]:
malaya_speech.config.speakernet_featurizer_config

{'sample_rate': 16000,
 'frame_ms': 20,
 'stride_ms': 10,
 'num_feature_bins': 64,
 'preemphasis': 0.97,
 'normalize_signal': True,
 'normalize_feature': True,
 'n_fft': 512,
 'frame_splicing': 1,
 'dither': 1e-05}

In [21]:
featurizer = malaya_speech.featurization.SpeakerNetFeaturizer(
    **malaya_speech.config.speakernet_featurizer_config
)